In [12]:
import torch as th
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
def task_df(vfs_filepath, gt_filepath, zone_filepath, task_name):
    vfs_robs = th.load(vfs_filepath)
    gt_robs = th.load(gt_filepath)
    zone_gt = th.load(zone_filepath)

    device = th.device('cpu')
    vfs_robs = vfs_robs.flatten().to(device).numpy()
    gt_robs = gt_robs.flatten().to(device).numpy()
    zone_gt = np.array(zone_gt)

    task_names = np.repeat(task_name, len(zone_gt))

    combined = np.stack((vfs_robs, gt_robs, zone_gt, task_names), axis=1)
    column_vales = ['vfs_robs', 'gt_robs', 'zone_gt', task_name]

    df = pd.DataFrame(data=combined, columns=column_vales)
    return df


In [25]:
# construct dataframe with [vfs_robs, gt_robs, zone_gt]
chain_vfs_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/vfs_robs_mcts_mpc_chain_expolre1_20240322_070715.pt'
chain_gt_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/ground_truth_mcts_mpc_chain_expolre1_20240322_070715.pt'
chain_zone_path = '/app/vfstl/src/VFSTL/controller_evaluation_result/zone_truth_mcts_mpc_chain_expolre1_20240322_070715.pt'

df = task_df(chain_vfs_path, chain_gt_path, chain_zone_path, 'chain')


df

chain
100


,vfs_robs,gt_robs,zone_gt,chain
0,0.10751193761825562,0.215563813822508,True,chain
1,-0.03746539354324341,-0.6065863684846197,True,chain
2,-0.16290265321731567,-1.8313955873555687,False,chain
3,0.04636865854263306,-0.4125655129986593,False,chain
4,0.059913456439971924,-0.14982707748834279,True,chain
...,...,...,...,...
95,0.06529015302658081,-0.009592985838032364,False,chain
96,0.11631810665130615,0.13840861843066188,True,chain
97,0.022017061710357666,-0.2286992138568188,False,chain
98,0.06911265850067139,0.010898671446205599,False,chain


In [ ]:
sns.boxplot(data=df, )